In [ ]:
# not every library is required for each day, but they are all loaded here
MAPBOX_TOKEN = 'enter token' #Optional for additional basemaps https://www.mapbox.com/

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

import numpy as np
import osmnx as ox
import pydeck as pdk
import json
from shapely.geometry import Polygon
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from PIL import Image
import networkx as nx
import contextily as ctx
from mpl_toolkits.basemap import Basemap
import rasterio

px.set_mapbox_access_token(MAPBOX_TOKEN)

In [ ]:
# Day 8 (and 6) - HDX data and rasters (Biomass Production in West Africa)
# https://data.humdata.org/dataset/acf_biomass_west-africa_raster

xmin, xmax, ymin, ymax = -17.5, 24, 1.75, 27.23
# Read the raster data
with rasterio.open('data/BiomassValue2024.tif') as src:
    raster_data = src.read(1)
    raster_transform = src.transform
    
# Create a basemap
fig, ax = plt.subplots(figsize=(10, 8))
m = Basemap(epsg=4326, 
            llcrnrlat=ymin, urcrnrlat=ymax, 
            llcrnrlon=xmin, urcrnrlon=xmax, 
            resolution='i', ax=ax)

# Fill continents with a specified color
m.shadedrelief()

# Plot the raster
plt.imshow(np.ma.masked_where(raster_data < 0, raster_data), cmap='YlGn', interpolation='nearest', extent=[xmin, xmax, ymin, ymax], vmin=0, vmax=49909)
cbar=plt.colorbar(label='Biomass Value (kg/ha)', fraction=0.025, pad=0.04)
# Add basemap features (optional)
m.drawcoastlines()
m.drawcountries()

# Set title
plt.title('Biomass Production in West Africa (2024)')
plt.savefig('maps/8_HDX.png', dpi=300, bbox_inches='tight')

In [ ]:
# Day 7 - Vintage (Lake Champlain Nautical Map)
# https://geodata.vermont.gov/datasets/758f2b40aa6a40f78ef8192cb60d35ec_21/explore
fig=go.Figure(go.Choroplethmapbox())
fig.update_layout(mapbox_style = 'carto-positron',
                mapbox_accesstoken=MAPBOX_TOKEN,
                height = 750,
                autosize=True,
                margin={"r":0,"t":0,"l":0,"b":0},
                paper_bgcolor='#303030',
                plot_bgcolor='#303030',
                mapbox=dict(center=dict(lat=44.5333, lon=-73.2), zoom=10),
                )

# with open('data/champlain.geojson', 'r') as file:
#     data = json.load(file)
gdf = gpd.read_file("data/champlain.geojson")
gdf = gdf.to_crs('epsg:4326')
# Filled polygon (chloropleth)
fig.add_trace(go.Choroplethmapbox(geojson=json.loads(gdf.to_json()), locations=gdf.index, z=[0.7],
                                    colorscale="blues", zmin=0, zmax=1,
                                    marker_opacity=0.3, marker_line_width=0, showlegend=False, showscale=False,))

waves = [[0.22, 0.6, 0.15], 
         [0.25, 0.15, 0.12], 
         [0.35, 0.7, 0.12],
         [0.17, 0.92, 0.1]]
for wave in waves:               
    fig.add_layout_image(
            dict(
                source=f"data/wave.png",
                xref="paper", yref="paper",
                x=wave[0], y=wave[1],
                sizex=wave[2], sizey=wave[2],
                sizing="contain",
                opacity=0.8,
                layer="above"
            ))

fig.add_layout_image(
            dict(
                source=f"data/lake_monster.png",
                xref="paper", yref="paper",
                x=0.33, y=0.37,
                sizex=0.15, sizey=0.15,
                sizing="contain",
                opacity=1,
                layer="above"
            ))
fig.add_layout_image(
            dict(
                source=f"data/compass.png",
                xref="paper", yref="paper",
                x=0.01, y=0.25,
                sizex=0.2, sizey=0.2,
                sizing="contain",
                opacity=1,
                layer="above"
            ))
fig.add_layout_image(
            dict(
                source=f"data/ship.png",
                xref="paper", yref="paper",
                x=0.18, y=0.73,
                sizex=0.15, sizey=0.15,
                sizing="contain",
                opacity=1,
                layer="above"
            ))
fig.add_shape(
    type="rect",
    xref="x",
    yref="y",
    x0=-74,
    y0=44,
    x1=-73,
    y1=45,
    fillcolor="tan",
    opacity=0.3,
    line=dict(width=0)
)
fig.add_annotation(
    x=-73.2,
    y=44.9,
    text="Lake Champlain",
    showarrow=False,
    font=dict(
            family="Old Standard TT, Courier New, monospace",
            size=40,)
)

fig.update_layout(height=600, width=1000, plot_bgcolor="#dfdfdf")

In [ ]:
# Day 6 - Rasters - See day 8!

In [ ]:
# Day 5 - A journey (New York State Election Polling Place Distance)
# data sources:
# https://data.gis.ny.gov/
# https://www.arcgis.com/home/item.html?id=23a056702cfd40848deef9597945a998#data

# Workflow inspiration - https://github.com/nhsx/nhs_time_of_travel

# Read in csv file
df = pd.read_csv("data/NYS_Elections_Districts_and_Polling_Locations_5446017761699402183.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

place_name = "New York, NY, USA"
# Fetch the street network
G = ox.graph_from_place(place_name, network_type='drive')
# Optionally, save the graph for later use
ox.save_graphml(G, "data/NY_street_network.graphml") 

#load the graph
G = ox.load_graphml("data/NY_street_network.graphml")

# Find the nearest node to each polling place
point_of_interest = ox.distance.nearest_nodes(G, X = gdf['Longitude'], Y=gdf['Latitude']) #---- X = longitude, Y = Latitude 

# Trip time in Mintues
#----------------------
trip_times = [2,5,10,15]

# speed in km/hourly
#-------------------------
travel_speed = 4.8

# add an edge attribute for time in minutes required to traverse each edge
#--------------------------------------------------------------------------
meters_per_minute = travel_speed * 1000 / 60 # convert speed in km per hour to meteres per minute as length attribute in edges sorted in metres and travel time specified in minutes
for _, _, _, data in G.edges(data=True, keys=True):  
    data['time'] = data['length'] / meters_per_minute
    
# Retrun one color for each isochrone (each of the trip times) 
#------------------------------------------------------------
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='bwr', start=0, stop =0.8, return_hex=True)[::-1]

# Color the edges according to isochrone then plot the urban network graph
#--------------------------------------------------------------------------
edge_colors = {}
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
    for point in point_of_interest:
        subgraph = nx.ego_graph(G, point, radius=trip_time, distance='time') #---- uses the edge attribute 'time' to define the isochrones for each of the trips
        for edge in subgraph.edges():
            edge_colors[edge] = color
nc = [edge_colors[edge] if edge in edge_colors else '#ff0000ff' for edge in G.edges()]
ns = [1 if edge in edge_colors else 1 for edge in G.edges()] #------ sets the size of the coloured edges at '10', while uncoloured edges (outside trip times) are not visualised
fig, ax = ox.plot_graph(G, node_size=0, edge_color=nc, edge_linewidth=ns, edge_alpha=0.8, figsize = (15,15), bgcolor="lightgrey")

ax.text(0.01, 0.9, 'New York City \nPolling Place Proximity', transform=ax.transAxes, ha="left", color='black', fontsize=30, fontfamily='Times New Roman', fontweight='bold')

patch1 = mpatches.Patch(color='#0000ffff', label='2')
patch2 = mpatches.Patch(color='#8888ffff', label='5')
patch3 = mpatches.Patch(color='#ffeeeeff', label='10')
patch4 = mpatches.Patch(color='#ff6666ff', label='15')
patch5 = mpatches.Patch(color='#ff0000ff', label='>20')

ax.legend(handles=[patch1,patch2,patch3,patch4,patch5], title='Walking Time [min]')

fig.savefig('maps/5_Journey.png', dpi=300, bbox_inches='tight')

In [ ]:
# Day 4 - Hexagons (US Tornado Occurrence)
# data sources: https://www.spc.noaa.gov/wcm/#data

# Read the CSV file
df = pd.read_csv("data/1950-2023_all_tornadoes.csv")
# remove data with slat > 48
df = df[df['slat']< 48]

fig = ff.create_hexbin_mapbox(
    data_frame=df, lat="slat", lon="slon",
    nx_hexagon=200, opacity=1, labels={"color": "Tornado Count"},
    min_count=0, color_continuous_scale="portland",
    mapbox_style = 'outdoors',
    # show_original_data=True, 
    # original_data_marker=dict(size=3, opacity=0.3, color="deeppink") # Cool option, but too much raw data for this example
)

gdf = gpd.GeoDataFrame({
    'customdata': fig.data[0]['customdata'].tolist(),
    'id':[item['id'] for item in fig.data[0]['geojson']['features']],
    'geometry':[Polygon(item['geometry']['coordinates'][0]) for item in fig.data[0]['geojson']['features']]
})
gdf.set_crs(epsg=4326, inplace=True)

gdf_poly = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
gdf_poly = gdf_poly.drop('name', axis = 1)
USA_gdf_area = gdf_poly[gdf_poly['iso_a3'] == 'USA'].reset_index(drop = True)

hexbins_in_USA = sjoin(gdf, USA_gdf_area, how='inner')

def get_coordinates(polygon):
    return [[list(i) for i in polygon.exterior.coords]]

hexbins_in_USA['coordinates'] = hexbins_in_USA['geometry'].apply(lambda x: get_coordinates(x))

## create a new geojson that matches the structure of fig.data[0]['geojson']['features']
new_geojson = [{
    'type': 'Feature', 
    'id': id, 
    'geometry': {
        'type': 'Polygon', 
        'coordinates': coordinate
    }
} for id, coordinate in zip(hexbins_in_USA['id'],hexbins_in_USA['coordinates'])]

fig.data[0]['geojson']['features'] = new_geojson
fig.data[0]['customdata'] = hexbins_in_USA['customdata']

# center map on US
fig.update_layout(mapbox=dict(center=dict(lat=39,lon=-95), zoom=2.9))

fig.update_layout(width=1000, height=600)
fig.write_image('maps/4_hexagons.png')

In [ ]:
# Day 3 - Polygons (Vermont Maple Production)
# data sources: https://www.nass.usda.gov/Publications/AgCensus/2022/Full_Report/Volume_1,_Chapter_2_County_Level/Vermont/st50_2_037_038.pdf
#               https://geodata.vermont.gov/datasets/VCGI::vt-data-county-boundaries-1/about

# Read the CSV file
df = pd.read_csv("data/maple_production.csv")

# Create a GeoDataFrame from geojson data 
gdf = gpd.read_file("data/VT_county.geojson")

# Join data
gdf = gdf.merge(df, how='left', left_on='CNTYNAME', right_on='county')

# feature engineering for pydeck viz,  scale production values 0-1
min_max_scaler = preprocessing.MinMaxScaler()
x = gdf["gallons"].values.reshape(-1, 1)
x_scaled = min_max_scaler.fit_transform(x)
gdf["area_norm"] = pd.Series(x_scaled.flatten())

# format data for use in pydeck
json_out = json.loads(gdf.to_json())
# inspect the first authority
json_out["features"][0]["properties"]
r = "250"
g = "(1 - properties.area_norm) * 255"
b = "properties.area_norm * 255"
fill = f"[{r},{g},{b}]"
geojson = pdk.Layer(
        "GeoJsonLayer",
        json_out,
        pickable=True,
        opacity=1,
        stroked=True,
        filled=True,
        extruded=True,
        wireframe=True,
        auto_highlight=True,
        get_elevation="properties.area_norm * 200",
        elevation_scale=100,
        get_fill_color=fill,
    )
tooltip = {"text": "{county}\n{gallons} gallons"}
view_state = pdk.ViewState(
    longitude=-72.7,
    latitude=43.5,
    zoom=7,
    max_zoom=15,
    pitch=50,
    bearing=2,
)
r = pdk.Deck(
    layers=geojson,
    initial_view_state=view_state,
    tooltip=tooltip,
)
r.to_html("maps/3_polygons_map.html")

# Create and save custom colorbar for final map
# Define your RGB color values
colors = [(1, 1, 0), (1, 0.5, 0.5), (1, 0, 1)]  # Red, Green, Blue

# Create a colormap using the defined colors
cmap = mcolors.LinearSegmentedColormap.from_list("my_cmap", colors)
mat = np.random.random((10,10))*1000000
plt.imshow(mat, origin="lower", cmap=cmap, interpolation='nearest')
cbar = plt.colorbar()
cbar.set_label('Gallons Maple Syrup Produced')
# Save the figure
plt.savefig('maps/3_polygons_colorbar.png', dpi=300, bbox_inches='tight')

In [ ]:
# Day 2 - Lines (City street network contrast map)
# data source: openstreetmap

# Select city and crs, note: larger cities take a few minutes
cityname = 'Portland, OR, USA'
crs = 4326

# Select color contrast
color = ['black', 'pink'] # text, background

# Get graph by geocoding
graph = ox.graph_from_place(cityname, network_type="walk")

# Project graph
graph = ox.projection.project_graph(graph, to_crs=crs)

# everything to gdfs
nodes, edges = ox.graph_to_gdfs(graph)

# Setup plot
fig, ax = plt.subplots(figsize=(10,8), dpi=200)
ax.set_axis_off()
ax.set_aspect('equal')
fig.set_facecolor(color[1])

# Plot data
edges.plot(
    ax=ax,
    color=color[0],
    linewidth=0.5
)

# plot city name
plt.annotate(cityname, xy=(0.5, 0), ha='center', xycoords='axes fraction', fontsize=20, color=color[0], weight='bold', family='monospace')

# Tight layout
plt.tight_layout()

# save figure
plt.savefig('maps/2_lines_'+cityname.split()[0][:-1]+'.png', dpi=300, bbox_inches='tight')

In [ ]:
# Day 1 - Points
# data source
# https://www.doogal.co.uk/FootballStadiumsCSV.ashx

# Read the CSV file
df = pd.read_csv("data/stadiums.csv")

# Create a GeoDataFrame from the CSV data
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Draw the map background
fig = plt.figure(figsize=(8, 8), dpi=150)
m = Basemap(projection='lcc', resolution='f',
            width=0.8E6, height=1.1E6, 
            lat_0=54.4, lon_0=-3,)
m.etopo(scale=1, alpha=0.5)

# Scatter stadium data with size reflecting capacity
m.scatter(gdf.Longitude, gdf.Latitude, latlon=True,
          c='goldenrod', s=(30*gdf.Capacity / gdf.Capacity.max())**2,
          alpha=0.7, edgecolors='orangered')

# Make legend with dummy points
for a in [10000, 30000, 50000, 70000]:
    plt.scatter([], [], c='k', alpha=0.7, s=(30*a / gdf.Capacity.max())**2,
                label=str(int(a/1000)) + 'k')
plt.legend(title='Capacity', scatterpoints=1, frameon=True,
           labelspacing=1.2, loc='lower left');
plt.title("UK Football Stadium Capacity", fontdict={'family': 'serif', 'size': 16})

# Add large cities to map
# Add a point with a label
cities = [[-0.1278, 51.5074, '  London'],
          [-2.244644, 53.483959, '  Manchester'],
          [-1.898575, 52.489471, '  Birmingham'],
          [-4.251433, 55.860916, '  Glasgow'],
          [-1.600000, 54.966667, '  Newcastle'],]

for city in cities:
    # Convert lat/lon to map coordinates
    x, y = m(city[0], city[1])
    m.plot(x, y, '.', c='black', markersize=8)
    plt.text(x, y, city[2], fontsize=14, alpha=1, c='black', verticalalignment='center')

x, y, arrow_length = 0.9, 0.9, 0.1
plt.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords='axes fraction')

# save figure
plt.savefig('maps/1_points.png', dpi=300, bbox_inches='tight')